In [4]:
import os
os.chdir('/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/src/vendor_run/mellotron')
from hparams import create_hparams
from model import Tacotron2
import torch
from loss_function import Tacotron2Loss
from train import prepare_dataloaders
import waveglow.glow
from waveglow.denoiser import Denoiser
import IPython.display as ipd
import sys
from text import cmudict, text_to_sequence
from data_utils import TextMelLoader, TextMelCollate
import waveglow
import librosa
from layers import TacotronSTFT
from train import warm_start_model
sys.path.append('waveglow/')
def load_mel(path):
    audio, sampling_rate = librosa.core.load(path, sr=hparams.sampling_rate)
    audio = torch.from_numpy(audio)
    if sampling_rate != hparams.sampling_rate:
        raise ValueError("{} SR doesn't match target {} SR".format(
            sampling_rate, stft.sampling_rate))
    audio_norm = audio.unsqueeze(0)
    audio_norm = torch.autograd.Variable(audio_norm, requires_grad=False)
    melspec = stft.mel_spectrogram(audio_norm)
    melspec = melspec.cuda()
    return melspec



arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')


In [5]:
hparams = create_hparams(None)

learning_rate = 1e-3#


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [6]:
# hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/todos_train.txt'
# hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/todos_val.txt'
hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_train.txt'
hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_val.txt'

hparams.batch_size = 2
hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
hparams.warm_start = True


def load_model(hparams):
    model = Tacotron2(hparams).cuda()
    if hparams.fp16_run:
        model.decoder.attention_layer.score_mask_value = finfo('float16').min

    return model


In [7]:
stft = TacotronSTFT(hparams.filter_length, hparams.hop_length, hparams.win_length,
                    hparams.n_mel_channels, hparams.sampling_rate, hparams.mel_fmin,
                    hparams.mel_fmax)

criterion = Tacotron2Loss()
train_loader, valset, collate_fn, train_sampler = prepare_dataloaders(hparams)
iteration = 0
epoch_offset = 0
is_overflow = False

i, batch = next(enumerate(train_loader))


torch.manual_seed(hparams.seed)
model = load_model(hparams)
model = warm_start_model(
    hparams.checkpoint_path, model, hparams.ignore_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)
optimizer.eps = 1e-16

Warm starting model from checkpoint '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'


In [9]:
arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')
audio_paths = hparams.validation_files#'data/examples_filelist.txt'
dataloader = TextMelLoader(audio_paths, hparams)
datacollate = TextMelCollate(1)
#file_idx = [0,1,2,3]
file_idx = 0
audio_path, text, sid = dataloader.audiopaths_and_text[0]
text = "its like this and like that and like this and uh"
def to_arpabet(text):
    from g2p_en import G2p
    g2p = G2p()
    return " ".join(
        [
            f"{{ {s.strip()} }}" if s.strip() not in ",." else s.strip()
            for s in " ".join(g2p(text)).split("  ")
        ]
    )
text = to_arpabet(text)
# get audio path, encoded text, pitch contour and mel for gst
text_encoded = torch.LongTensor(text_to_sequence(text, hparams.text_cleaners, arpabet_dict))[None, :].cuda()    
pitch_contour = dataloader[file_idx][3][None].cuda()
mel = load_mel(audio_path)

In [12]:
waveglow_path = '../../vendor_tools/waveglow/waveglow_256channels_universal_v4.pt'
waveglow = torch.load(waveglow_path)['model'].cuda().eval()
denoiser = Denoiser(waveglow).cuda().eval()
#from glow import infer

In [13]:
speaker_ids = torch.tensor([0], device='cuda:0').to(torch.int64)
#xnew = (text_encoded, 0,speaker_ids,x[6][[0]])
xnew =(text_encoded, 0,speaker_ids,torch.zeros([1, 1, 481], device =  'cuda:0'))
with torch.no_grad():
    model.zero_grad()
    mel_outputs, mel_outputs_postnet, gate_outputs, rhythm = model.inference(xnew)
    rhythm = rhythm.permute(1, 0, 2)
    audio = denoiser(waveglow.infer(mel_outputs_postnet, sigma=0.8), 0.01)[:, 0]

Warning! Reached max decoder steps


In [14]:
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

In [2]:
from numba import cuda 
import torch
#cuda.select_device(0)
#cuda.close()
cuda.select_device(0)
cuda.close()
torch.cuda.empty_cache()